#### Build a Simple LLM Application with LCEL¶
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

Using language models

Using PromptTemplates and OutputParsers

Using LangChain Expression Language (LCEL) to chain components together

Debugging and tracing your application using LangSmith

Deploying your application with LangServe

In [1]:
# OpenAI API Key and Open source model(llama3,gemma2, mistral--Groq) 

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_openai import ChatOpenAI    #b for open ai
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fa86132f040>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fa86132fa30>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage    

messages = [
    SystemMessage(content="Translate the following from English to French"),    # it instructs llm model how it should behave   here it is acting like translator
    HumanMessage(content="Hello, How are you?")     # this is the question human asks
]

result = model.invoke(messages)
result

AIMessage(content='Bonjour, comment allez-vous ? \n', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 27, 'total_tokens': 38, 'completion_time': 0.022, 'prompt_time': 0.002915339, 'queue_time': None, 'total_time': 0.024915338999999998}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9330f8bc-b102-4d44-9e9e-802e93334925-0', usage_metadata={'input_tokens': 27, 'output_tokens': 11, 'total_tokens': 38})

In [10]:
# we only want the output message
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ? \n'

In [11]:
# LCEL=lANGCHAIN EXPRESSION LANGUAGE
# using LCEL we can change the components

chain = model|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n\n\n You can also say:\n\n* Salut, comment vas-tu ? (More informal) \n\n'

In [12]:
# prompt template

from langchain_core.prompts import ChatPromptTemplate    

generic_template = "Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [19]:
results = prompt.invoke({"language":"French","text":"Hello"})
results
# this gives the same one as above

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French:'), HumanMessage(content='Hello')])

In [18]:
results.to_messages()

[SystemMessage(content='Translate the following into French:'),
 HumanMessage(content='Hello')]

In [20]:
# chaining together components with LCEL
chain = prompt|model|parser
chain.invoke({"language":"french","text":"Hello"})

'Bonjour  \n'